# Analysis of Wind Data in Ireland

This notebook looks at weather data related to wind in Ireland.
The data was taken from Met Éireann and is used here to explore how wind speed changes over time and what that might mean for wind energy.

## About the Data

Before analysing the data, it is useful to understand where it comes from and what information it contains.

In [ ]:
import pandas as pd

skip_rows = 25

# Read CSV directly into working DataFrame
wind_simple = pd.read_csv("data/Wind_Speed_hourly.csv", skiprows=skip_rows)

# Rename only relevant columns
wind_simple.rename(columns={'date': 'Time', 'wdsp': 'Wind'}, inplace=True)


# Convert Time to datetime
wind_simple['Time'] = pd.to_datetime(wind_simple['Time'], format='%d-%b-%Y %H:%M')


wind_simple = wind_simple[['Time', 'Wind']]


print(wind_simple.head())
print(wind_simple.info())

/tmp/ipykernel_3768/2162410776.py:6: DtypeWarning: Columns (8,9,14,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  wind_simple = pd.read_csv("data/Wind_Speed_hourly.csv", skiprows=skip_rows)


KeyError: 'Time'

## Preparing the Dataset

The dataset contains some missing and inconsistent values which need to be fixed before doing any analysis.

## Exploring Wind Speed

In this section, different summaries and plots are used to explore wind speed patterns over time.

## Discussion of Results

This section discusses what was observed from the analysis and how it may relate to wind energy.